# LLM Prompting Research Tool - Simplified

This notebook tests English and Spanish prompts with configurable step count and education level for MyHeartCounts nudge generation.

In [ ]:
import json
import requests
from google.colab import userdata
from typing import Dict, Any

## Configuration

**Edit these values to test different scenarios:**

In [ ]:
# USER CONFIGURATION - Edit these values
STEP_COUNT = 5000                    # Daily step count to include in prompts
EDUCATION_LEVEL = "Bachelor's degree" # Education level to include in prompts
LANGUAGE = 'en'                      # 'en' for English, 'es' for Spanish

# OpenAI Configuration
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
MODEL = 'gpt-3.5-turbo'
MAX_TOKENS = 1000
TEMPERATURE = 0.7

print(f"✅ Configuration set: {STEP_COUNT} steps, education: {EDUCATION_LEVEL}, language: {LANGUAGE}")
print("✅ API key configured")

## LLM Interface

In [ ]:
class LLMTester:
    def __init__(self, api_key: str, model: str = MODEL, max_tokens: int = MAX_TOKENS, temperature: float = TEMPERATURE):
        self.api_key = api_key
        self.model = model
        self.max_tokens = max_tokens
        self.temperature = temperature

    async def call_openai_api(self, prompt: str) -> Dict[str, Any]:
        try:
            response = requests.post(
                'https://api.openai.com/v1/chat/completions',
                headers={
                    'Authorization': f'Bearer {self.api_key}',
                    'Content-Type': 'application/json'
                },
                json={
                    'model': self.model,
                    'messages': [{'role': 'user', 'content': prompt}],
                    'max_tokens': self.max_tokens,
                    'temperature': self.temperature
                },
                timeout=30
            )

            if not response.ok:
                raise Exception(f'OpenAI API error: {response.status_code} {response.text}')

            data = response.json()
            content = data['choices'][0]['message']['content']
            return {'success': True, 'content': content, 'error': None}

        except Exception as error:
            return {'success': False, 'content': None, 'error': str(error)}

    def validate_response(self, content: str) -> Dict[str, Any]:
        """Validate LLM response format"""
        try:
            parsed_nudges = json.loads(content)
            
            if not isinstance(parsed_nudges, list):
                return {'is_valid': False, 'error': 'Response is not an array'}
            
            if len(parsed_nudges) != 7:
                return {'is_valid': False, 'error': f'Expected 7 items, got {len(parsed_nudges)}'}
            
            for i, nudge in enumerate(parsed_nudges):
                if not isinstance(nudge, dict):
                    return {'is_valid': False, 'error': f'Item {i} is not an object'}
                if 'title' not in nudge or 'body' not in nudge:
                    return {'is_valid': False, 'error': f'Item {i} missing title or body'}
                if not nudge['title'].strip() or not nudge['body'].strip():
                    return {'is_valid': False, 'error': f'Item {i} has empty title or body'}
            
            return {'is_valid': True, 'parsed_data': parsed_nudges, 'error': None}
            
        except json.JSONDecodeError as e:
            return {'is_valid': False, 'error': f'Invalid JSON: {str(e)}'}

    async def test_prompt(self, prompt: str) -> Dict[str, Any]:
        """Test a prompt and return results"""
        print(f"🧪 Testing prompt...")
        print(f"📝 Prompt length: {len(prompt)} characters")
        
        # Call API
        api_result = await self.call_openai_api(prompt)
        
        if api_result['success']:
            print("✅ API call successful")
            
            # Validate response
            validation = self.validate_response(api_result['content'])
            
            if validation['is_valid']:
                print("✅ Response validation passed")
                return {
                    'success': True,
                    'nudges': validation['parsed_data'],
                    'raw_response': api_result['content']
                }
            else:
                print(f"❌ Response validation failed: {validation['error']}")
                return {
                    'success': False,
                    'error': validation['error'],
                    'raw_response': api_result['content']
                }
        else:
            print(f"❌ API call failed: {api_result['error']}")
            return {
                'success': False,
                'error': api_result['error'],
                'raw_response': None
            }

# Initialize tester
tester = LLMTester(OPENAI_API_KEY)
print("🚀 LLM Tester initialized")

## Prompts with Step Count and Education Level

**Edit these prompts to test different approaches:**

In [ ]:
def get_english_prompt(step_count: int, education_level: str) -> str:
    """Edit this English prompt to test different approaches"""
    return f"""Generate 7 motivational sports and exercise nudges for a heart health study participant.

Participant information:
- Recent daily step count average: {step_count}
- Education level: {education_level}

Each nudge should:
- Be encouraging and positive
- Focus on different types of physical activities and sports
- Be personalized and engaging based on the participant's step count and education level
- Include a clear call to action
- Be suitable for someone in a heart health study
- Incorporate step count references when relevant
- Adapt language and suggestions to the participant's education level

Return the response as a JSON array with exactly 7 objects, each having "title" and "body" fields.
Example format:
[
  {{"title": "Morning Energy Boost", "body": "Start your day with a 15-minute walk! Your heart will love the gentle cardio."}},
  ...
]

Make each nudge unique and focus on different activities like walking, swimming, dancing, team sports, strength training, yoga, etc."""

def get_spanish_prompt(step_count: int, education_level: str) -> str:
    """Edit this Spanish prompt to test different approaches"""
    return f"""Genera 7 recordatorios motivacionales de deportes y ejercicio para un participante en un estudio de salud cardíaca.

Información del participante:
- Promedio de pasos diarios recientes: {step_count}
- Nivel educativo: {education_level}

Cada recordatorio debe:
- Ser alentador y positivo
- Enfocarse en diferentes tipos de actividades físicas y deportes
- Ser personalizado y atractivo basado en el conteo de pasos y nivel educativo del participante
- Incluir una llamada clara a la acción
- Ser adecuado para alguien en un estudio de salud cardíaca
- Incorporar referencias al conteo de pasos cuando sea relevante
- Adaptar el lenguaje y las sugerencias al nivel educativo del participante

Devuelve la respuesta como un array JSON con exactamente 7 objetos, cada uno con campos "title" y "body".
Formato de ejemplo:
[
  {{"title": "Impulso de Energía Matutino", "body": "¡Comienza tu día con una caminata de 15 minutos! Tu corazón amará el cardio suave."}},
  ...
]

Haz cada recordatorio único y enfócate en diferentes actividades como caminar, nadar, bailar, deportes de equipo, entrenamiento de fuerza, yoga, etc."""

print("📝 Prompt functions defined")

## Run Test

**Execute this cell to test your configured prompt:**

In [ ]:
# Generate prompt based on configuration
if LANGUAGE == 'en':
    prompt = get_english_prompt(STEP_COUNT, EDUCATION_LEVEL)
    print(f"🇺🇸 Testing English prompt with {STEP_COUNT} steps, education: {EDUCATION_LEVEL}")
elif LANGUAGE == 'es':
    prompt = get_spanish_prompt(STEP_COUNT, EDUCATION_LEVEL)
    print(f"🇪🇸 Testing Spanish prompt with {STEP_COUNT} steps, education: {EDUCATION_LEVEL}")
else:
    print("❌ Invalid language. Use 'en' or 'es'")
    prompt = None

if prompt:
    # Print full prompt for debugging
    print("\n" + "="*80)
    print("📤 FULL PROMPT SENT TO LLM:")
    print("="*80)
    print(prompt)
    print("="*80)
    
    # Test the prompt
    result = await tester.test_prompt(prompt)
    
    # Print full LLM response for debugging
    print("\n" + "="*80)
    print("📥 FULL LLM RESPONSE:")
    print("="*80)
    if result['raw_response']:
        print(result['raw_response'])
    else:
        print("No response received")
    print("="*80)
    
    # Display results
    if result['success']:
        print("\n📋 Generated Nudges:")
        for i, nudge in enumerate(result['nudges'], 1):
            print(f"{i}. {nudge['title']}")
            print(f"   {nudge['body']}\n")
    else:
        print(f"\n❌ Test failed: {result['error']}")
        print("Check the full LLM response above for details.")